# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
# Описание данных
children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию
Импортируем библиотеку pandas и прочитаем файл с исходными данными

In [1]:
import pandas as pd 
from pymystem3 import Mystem 
m = Mystem()
from collections import Counter 
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/datasets/data.csv')

Изучим общую информацию о данных

In [3]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Выведем название столбцов

In [4]:
data.columns 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [5]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посмотрим пропуски

In [6]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Посмотрим полные дубликаты

In [7]:
data.duplicated().sum()

54

Рассмотрим уникальные значения данных для каждого столбца

In [8]:
for row in data: 
    print(data[row].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее   

**Вывод** 

Общее количество значений в данных 21525. В таблице 12 столбцов.
В столбце children опечатки (-1 и 20 детей);
В столбце days_employed отрицательные значения и пропуски;
dob_years-нулевые значения;
education-дубликаты;
gender-пропуск;
total_income-пропуски;
purpose-дубликаты.
Наша задача исправить эти недочеты,не теряя информацию.

## Шаг 2. Предобработка данных

### Обработка пропусков
Меняем 20 детей на 2 и -1 ребенка на 1

In [9]:
data['children'] = data['children'].replace(20, 2) 
data['children'] = data['children'].replace(-1, 1) 

Удаляем 'XNA' 

In [10]:
data = data[data['gender'] != 'XNA'] 

Высчитываем медиану в столбце dob_years

In [11]:
median_years = data['dob_years'].median() 
median_years

42.0

Меняем нули на высчитанную медиану

In [12]:
data['dob_years'] = data['dob_years'].replace(0, 42) 

In [13]:
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Меняем пропуски на нули

In [14]:
data['total_income'] = data['total_income'].fillna(0) 

Выведем уникальные значения

In [15]:
data['income_type'].value_counts() 

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Медиана для total_income по столбцу income_type

In [16]:
data.groupby('income_type')['total_income'].median() 

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        139034.452386
компаньон          162379.674930
пенсионер          110179.690761
предприниматель    249581.572474
сотрудник          133546.457238
студент             98201.625314
Name: total_income, dtype: float64

Расчитаем медиану в total_income по категориям в income_type

In [17]:
data.loc[(data['total_income'] == 0) & (data['income_type'] == 'безработный'), 'total_income'] = \
                                        data.groupby('income_type')['total_income'].median()[0]

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'в декрете'), 'total_income'] = \
                                        data.groupby('income_type')['total_income'].median()[1]

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'госслужащий'), 'total_income'] = \
                                        data.groupby('income_type')['total_income'].median()[2]

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'компаньон'), 'total_income'] = \
                                        data.groupby('income_type')['total_income'].median()[3]

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'пенсионер'), 'total_income'] = \
                                        data.groupby('income_type')['total_income'].median()[4]

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'предприниматель'), 'total_income'] = \
                                        data.groupby('income_type')['total_income'].median()[5]

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'сотрудник'), 'total_income'] = \
                                        data.groupby('income_type')['total_income'].median()[6]

data.loc[(data['total_income'] == 0) & (data['income_type'] == 'студент'), 'total_income'] = \
                                        data.groupby('income_type')['total_income'].median()[7]
data.head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Меняем пропуски на нули

In [18]:
data['days_employed'] = data['days_employed'].fillna(0) 

Выведем уникальные значения

In [19]:
data['children'].value_counts() 

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Медиана для days_employed по столбцу children

In [20]:
data.groupby('children')['days_employed'].median() 

children
0    -795.578884
1   -1206.986715
2   -1382.477400
3   -1344.473971
4   -1696.862177
5    -773.124856
Name: days_employed, dtype: float64

Расчитаем медиану в days_employed по категориям в children

In [21]:
data.loc[(data['days_employed'] == 0) & (data['children'] == '0'), 'days_employed'] = \
                                        data.groupby('children')['days_employed'].median()[0]

data.loc[(data['days_employed'] == 0) & (data['children'] == '1'), 'days_employed'] = \
                                        data.groupby('children')['days_employed'].median()[1]

data.loc[(data['days_employed'] == 0) & (data['children'] == '2'), 'days_employed'] = \
                                        data.groupby('children')['days_employed'].median()[2]

data.loc[(data['days_employed'] == 0) & (data['children'] == '3'), 'days_employed'] = \
                                        data.groupby('children')['days_employed'].median()[3]

data.loc[(data['days_employed'] == 0) & (data['children'] == '4'), 'days_employed'] = \
                                        data.groupby('children')['days_employed'].median()[4]

data.loc[(data['days_employed'] == 0) & (data['children'] == '5'), 'days_employed'] = \
                                        data.groupby('children')['days_employed'].median()[5]
data.head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод** 

Поменяли значения в столбце дети,так как скорей всего это просто опечатки или "заглушки".В столбце gender убрали пропущенное значение,так как оно одно и ни на что не повлияет.В столбце dob_years 101 нулевые значения(это не существенно) заменили на медиану.Что бы заменить нули в столбце total_income взяли медиану для каждой категории по столбцу income_type.Взяли по этому столбцу,потому что доход на прямую зависит от типа занятости.Заменили нулевые значения в days_employed на медиану категорий в children.Предположили,что стаж может зависить от детей и их количества(декреты,больничные).

### Замена типа данных

In [22]:
data['total_income'] = data['total_income'].astype('int') 
data['days_employed'] = data['days_employed'].astype('int') 

Переводим в положительные

In [23]:
data['days_employed'] = abs(data['days_employed']) 
data.head() 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод** 

В столбце total_income значения принимают Length: 19351, dtype: int64.Переведем в int64,так как они нам нужны в дальнейшем для подсчетов.И в столбце days_employed были минусовые и дробные значения,перевели их в целые и положительные.Теперь такие числа можно считать,чем мы и займемся при выяснениях зависимости возврата кредита в срок.

### Обработка дубликатов

Привели все значения к одному регистру

In [24]:
data['education'] = data['education'].str.lower() 

Посмотрим сколько значений в каких категориях

In [25]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

Полные дубликаты таблицы

In [26]:
data.duplicated().sum()

72

Посмотрим сколько дубликатов в столбце

In [27]:
data.duplicated('education').sum()

21519

Удаляем дубликаты в таблице

In [28]:
data = data.drop_duplicates().reset_index(drop=True) 
data.duplicated().sum()

0

**Вывод**

Убрали дубликаты в столбце education с помощью нижнего регистра,так как категории-СРЕДНЕЕ=среднее=Среднее(по аналогии:высшее,неоконченное высшее,ученая степень,начальное)- обозначают одно и тоже. Иначе таблица не смогла бы нам передать достоверную информацию при анализе.Еще обнаружили 72 полных дубликата в таблице,после исправления дубликатов в столбце education,они не ушли,поэтому мы их удалили.

### Лемматизация
Леммантизируем слова в столбце purpose и убираем перенос в конце.Создаем столбец с леммами

In [29]:
def lem_purpose(purpose):
    lem = ''.join(m.lemmatize(purpose)[:-1]) 
    return lem
data['lem_purpose'] = data['purpose'].apply(lem_purpose)
data['lem_purpose'].value_counts()

автомобиль                                972
свадьба                                   791
на проведение свадьба                     767
сыграть свадьба                           765
операция с недвижимость                   675
покупка коммерческий недвижимость         661
операция с жилье                          652
покупка жилье для сдача                   651
операция с коммерческий недвижимость      650
жилье                                     646
покупка жилье                             646
покупка жилье для семья                   638
строительство собственный недвижимость    635
недвижимость                              633
операция со свой недвижимость             627
строительство жилой недвижимость          624
покупка свой жилье                        620
покупка недвижимость                      620
строительство недвижимость                619
ремонт жилье                              607
покупка жилой недвижимость                606
на покупка свой автомобиль        

Считает буквы

In [30]:
Counter(data['lem_purpose'].sum())

Counter({'п': 19244,
         'о': 49012,
         'к': 16729,
         'у': 7210,
         'а': 33479,
         ' ': 33567,
         'ж': 13004,
         'и': 39125,
         'л': 17666,
         'ь': 23444,
         'е': 32930,
         'р': 14267,
         'б': 11738,
         'т': 21046,
         'н': 21127,
         'в': 23876,
         'м': 15427,
         'д': 14191,
         'ы': 4602,
         'й': 8608,
         'з': 4917,
         'с': 26058,
         'г': 765,
         'ц': 2604,
         'я': 5435,
         'ч': 3276,
         'ф': 436})

Считает словосочетания

In [31]:
Counter(data['lem_purpose'])

Counter({'покупка жилье': 646,
         'приобретение автомобиль': 461,
         'дополнительный образование': 460,
         'сыграть свадьба': 765,
         'операция с жилье': 652,
         'образование': 447,
         'на проведение свадьба': 767,
         'покупка жилье для семья': 638,
         'покупка недвижимость': 620,
         'покупка коммерческий недвижимость': 661,
         'покупка жилой недвижимость': 606,
         'строительство собственный недвижимость': 635,
         'недвижимость': 633,
         'строительство недвижимость': 619,
         'на покупка подержать автомобиль': 478,
         'на покупка свой автомобиль': 505,
         'операция с коммерческий недвижимость': 650,
         'строительство жилой недвижимость': 624,
         'жилье': 646,
         'операция со свой недвижимость': 627,
         'автомобиль': 972,
         'заниматься образование': 408,
         'сделка с подержанный автомобиль': 486,
         'получение образование': 442,
         'свадьба': 79

Разбиваем на категории.Создадим новый столбец category с категориями на основании результатов лемматизации

In [32]:
def categorizer(lem_purpose): 
    if 'свадьба' in lem_purpose:
        return 'свадьба'
    elif ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose):
        return 'операции с недвижимостью'
    elif 'автомобиль' in lem_purpose:
        return 'автомобиль'
    else:
        return 'образование'
data['category'] = data['lem_purpose'].apply(categorizer) 
data['category'].value_counts()

операции с недвижимостью    10810
автомобиль                   4306
образование                  4013
свадьба                      2323
Name: category, dtype: int64

In [33]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,свадьба


**Вывод**

Создаём столбец lem_purpose, в котором каждая ячейка — это список лемматизированных слов из столбца purpose.И по нему отбираем наши цели на кредит по 4 категориям.В эти категории выбрали самые встречаемые словосочетания.

### Категоризация данных

In [34]:
data['children'].value_counts()

0    14089
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Разобьем на категории.Создадим столбец children_group,где дети разбиты на категории

In [35]:
def children_group(children): 
    if children == 0:
        return 'без детей'
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 ребенка'
    if children == 3:
        return '3 ребенка'
    if children == 4:
        return '4 ребенка'
    return '5 детей'
data['children_group'] = data['children'].apply(children_group)
data['children_group'].value_counts()

без детей    14089
1 ребенок     4855
2 ребенка     2128
3 ребенка      330
4 ребенка       41
5 детей          9
Name: children_group, dtype: int64

In [36]:
data['family_status_id'].value_counts()

0    12339
1     4149
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

Разобьем на категории.Создадим столбец family_status_id_group,где данные разбиты на категории

In [37]:
def family_status_id_group(family_status_id): 
    if family_status_id <= 1:
        return 'полная семья'
    return 'неполная семья'
data['family_status_id_group'] = data['family_status_id'].apply(family_status_id_group)
data['family_status_id_group'].value_counts()

полная семья      16488
неполная семья     4964
Name: family_status_id_group, dtype: int64

In [38]:
data['debt'].value_counts()

0    19711
1     1741
Name: debt, dtype: int64

Разобьем на категории.Создадим столбец debt_group,где данные разбиты на категории

In [39]:
def debt_group(debt): 
    if debt == 0:
        return 'задолжность'
    return 'нет задолжности'
data['debt_group'] = data['debt'].apply(debt_group)
data['debt_group'].value_counts()

задолжность        19711
нет задолжности     1741
Name: debt_group, dtype: int64

Количество задолжности по каждой группе

In [40]:
data.groupby('children_group')['debt'].sum()

children_group
1 ребенок     445
2 ребенка     202
3 ребенка      27
4 ребенка       4
5 детей         0
без детей    1063
Name: debt, dtype: int64

In [41]:
data.groupby('family_status_id_group')['debt'].sum()

family_status_id_group
неполная семья     422
полная семья      1319
Name: debt, dtype: int64

Посмотрим минимальное,максимальное,медиану и среднее значения

In [42]:
print(data['total_income'].min())
print(data['total_income'].max())
print(data['total_income'].median())
print(data['total_income'].mean())

20667
2265604
140215.5
164396.1652992728


Разобьем на категории.Создадим столбец total_income_group,где доходы разбиты на категории

In [43]:
def total_income_group(total_income): 
    if total_income <= 110000:
        return 'Доход меньше 110'
    if total_income <= 140000 and total_income > 110000:
        return 'Доход 110-140'
    if total_income <= 200000 and total_income > 140000:
        return 'Доход 140-200'
    if total_income > 200000:
        return 'Доход больше 200'
data['total_income_group'] = data['total_income'].apply(total_income_group) 
data['total_income_group'].value_counts()

Доход 140-200       5681
Доход меньше 110    5642
Доход больше 200    5066
Доход 110-140       5063
Name: total_income_group, dtype: int64

In [44]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose,category,children_group,family_status_id_group,debt_group,total_income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье,операции с недвижимостью,1 ребенок,полная семья,задолжность,Доход больше 200
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль,автомобиль,1 ребенок,полная семья,задолжность,Доход 110-140
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье,операции с недвижимостью,без детей,полная семья,задолжность,Доход 140-200
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование,образование,3 ребенка,полная семья,задолжность,Доход больше 200
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба,свадьба,без детей,полная семья,задолжность,Доход 140-200


**Вывод**

Разбили на категории(children_group, family_status_id_group, debt_group) по которым надо выяснить взаимосвязь возрата кредита в срок.Детей разбили по количеству,статус семьи на полные и неполные,зодолжность на была и не была. Видим,что у кредиторов с детьми реже возможность просрочки и в неполных семьях долгов меньше,чем в полных семьях.Разбили столбец total_income на категории так,что бы в каждой было приблизительно равное количество показателей для того,что бы аналитика была правдоподобнее.Клиенты с меньшим доходом берут реже кредит и у них реже просрочка по нему.Больше всего клиентов с доходом 100-150.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Количество задолжности по каждой группе

In [45]:
children_count_delay = data.groupby('children_group')['debt'].sum() 
children_count_delay

children_group
1 ребенок     445
2 ребенка     202
3 ребенка      27
4 ребенка       4
5 детей         0
без детей    1063
Name: debt, dtype: int64

Посчитаем уникальные значения в столбце children_group и присвоим к ним переменную children_total

In [46]:
children_total = data['children_group'].value_counts() 
children_total

без детей    14089
1 ребенок     4855
2 ребенка     2128
3 ребенка      330
4 ребенка       41
5 детей          9
Name: children_group, dtype: int64

Считаем зависимость между наличием детей и возвратом кредита в срок

In [47]:
children_debt_norm = children_count_delay / children_total *100 
children_debt_norm 

1 ребенок    9.165808
2 ребенка    9.492481
3 ребенка    8.181818
4 ребенка    9.756098
5 детей      0.000000
без детей    7.544893
dtype: float64

**Вывод**

Наблюдаем,что чем больше детей,тем выше шанс просрочки. Однако у тех клиентов у которых 5 детей просрочка равна нули.Это исключение,так как таких клиентов всего 9.Так же видем,что клиенты без детей чаще отдают кредит во время,чем с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Количество задолжности по каждой группе

In [48]:
family_status_count_delay = data.groupby('family_status_id_group')['debt'].sum() 
family_status_count_delay

family_status_id_group
неполная семья     422
полная семья      1319
Name: debt, dtype: int64

Посчитаем уникальные значения в столбце family_status_id_group и присвоим к ним переменную family_status_total

In [49]:
family_status_total = data['family_status_id_group'].value_counts()
family_status_total

полная семья      16488
неполная семья     4964
Name: family_status_id_group, dtype: int64

Считаем зависимость между семейным положением и возвратом кредита в срок

In [50]:
family_status_debt_norm = family_status_count_delay / family_status_total *100 
family_status_debt_norm

неполная семья    8.501209
полная семья      7.999757
dtype: float64

**Вывод**

В неполных семьях долгов по кредиту меньше,чем в полных,хотя разрыв между ними не большой,но даже такая небольшая разница может быть важной для банка.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Количество задолжности по каждой группе

In [51]:
total_income_count_delay = data.groupby('total_income_group')['debt'].sum() 
total_income_count_delay

total_income_group
Доход 110-140       431
Доход 140-200       497
Доход больше 200    358
Доход меньше 110    455
Name: debt, dtype: int64

Посчитаем уникальные значения в столбце total_income_group и присвоим к ним переменную total_income_new

In [52]:
total_income_new = data['total_income_group'].value_counts() 
total_income_new

Доход 140-200       5681
Доход меньше 110    5642
Доход больше 200    5066
Доход 110-140       5063
Name: total_income_group, dtype: int64

Считаем зависимость между уровнем дохода и возвратом кредита в срок

In [53]:
total_income_norm = total_income_count_delay / total_income_new * 100 
total_income_norm 

Доход 110-140       8.512739
Доход 140-200       8.748460
Доход больше 200    7.066719
Доход меньше 110    8.064516
dtype: float64

**Вывод**

Клиенты с доходом 140-200 чаще всего имеют долги.А с доходом больше 200 - меньше просрочки.Клиенты с доходом меньше 110 показатель просрочки выше,чем у клиентов с доход больше 200,но меньше,чем с доходом 140-200.

- Как разные цели кредита влияют на его возврат в срок?

Количество задолжности по каждой группе

In [54]:
category_count_delay = data.groupby('category')['debt'].sum()
category_count_delay

category
автомобиль                  403
образование                 370
операции с недвижимостью    782
свадьба                     186
Name: debt, dtype: int64

Посчитаем уникальные значения в столбце category и присвоим к ним переменную category_total

In [55]:
category_total = data['category'].value_counts()
category_total

операции с недвижимостью    10810
автомобиль                   4306
образование                  4013
свадьба                      2323
Name: category, dtype: int64

Считаем зависимость,как разные цели кредита влияют на его возврат в срок

In [56]:
category_debt_norm = category_count_delay / category_total * 100 
category_debt_norm

автомобиль                  9.359034
образование                 9.220035
операции с недвижимостью    7.234043
свадьба                     8.006888
dtype: float64

**Вывод**

Клиенты чаще возвращают кредит в срок,если он взят на операции с недвижимостью и на свадьбу.А на покупку авто и образования - просрочка чаще.

## Шаг 4. Общий вывод

Исправили таблицу,разбили на категории,которые могут влиять на вероятность выплаты кредита в срок и посмотрели какие критерии влияют на это. Клиенты с неполной семьей, без детей,с доходом больше 200 тысяч отличный вариант для банка.И лучше выдавать кредиты на такие цели,как 'операции с недвижемостью' и 'свадьба' так как они чаще возвращаются в срок.